In [1]:
import json
import numpy as np
import sys

In [2]:
plan = [{'days': '1',
  'breakfast': 'TwnzM8mJn_nT2PJf1x-9kQ',
  'morning_attractions': ['Ib9HV7ekw459jM1Ksdiyiw'],
  'lunch': 'wHkYLlZyPXllrQRlvidUlg',
  'afternoon_attractions': ['kbEVlzQLcYS3JSQPG9QMOQ',
   '8_O6LXLyMgpq1g9CIwcW4w'],
  'dinner': 'S8ZFYEgMejpChID8tzKo9A',
  'night_attractions': ['pxZAz8pv18wK_t-m8WpN0g'],
  'accommodation': '3QPAh9VvYNTnqAFgBeBcng'},
 {'days': '2',
  'breakfast': 'rYqmaOIULRouz_1db07OdQ',
  'morning_attractions': ['Qw7tz-UkPrpXaVidWuab4Q'],
  'lunch': '6ajnOk0GcY9xbb5Ocaw8Gw',
  'afternoon_attractions': ['4mWzXhD8vo0bABVCGAhlqA',
   'ytynqOUb3hjKeJfRj5Tshw'],
  'dinner': '05ev984NYfimRN0UiFrxaA',
  'night_attractions': ['dBRWKIS7h-qZCi8EjUJ2HA'],
  'accommodation': '3QPAh9VvYNTnqAFgBeBcng'}] 

with open('../Datasets/Attractions_task1.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

with open('../Datasets/Hotels_task1.jsonl', 'r') as f:
    data_hotel = [json.loads(line) for line in f]

def getCordinate(id):
    for attraction in data:
        if attraction['business_id'] == id:
            return (attraction['latitude'], attraction['longitude'])
def getCordinate_Hotel(id):
    for hotel in data_hotel:
        if hotel['business_id'] == id:
            return (hotel['latitude'], hotel['longitude'])

In [3]:
#get the cordinates
cordinates = []
for day in plan:
    cordinate_one_day = []
    
    cordinate_one_day.append(getCordinate_Hotel(day['accommodation']))
    
    for attraction in day['morning_attractions']:
        cordinate_one_day.append(getCordinate(attraction))
    for attraction in day['afternoon_attractions']:
        cordinate_one_day.append(getCordinate(attraction))
    for attraction in day['night_attractions']:
        cordinate_one_day.append(getCordinate(attraction))
            
    cordinates.append(cordinate_one_day)
cordinates

[[(39.946396818, -75.1609669488),
  (39.9488980012, -75.1500296367),
  (39.9495774962, -75.1503095688),
  (39.953893, -75.149068),
  (39.9445424967, -75.1414754636)],
 [(39.946396818, -75.1609669488),
  (39.965573, -75.180969),
  (39.942712, -75.159313),
  (39.9533414645, -75.1588545174),
  (39.9582109, -75.1731373822)]]

In [4]:
def getDistanceMatrix(cordinates):
    n = len(cordinates)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i][j] = distance_matrix[j][i] = ((cordinates[i][0]*1000 - cordinates[j][0]*1000)**2 + (cordinates[i][1]*1000 - cordinates[j][1]*1000)**2)**0.5
    return distance_matrix

In [5]:
cordinates_one_day = cordinates[0]

distance_matrix = getDistanceMatrix(cordinates_one_day)
distance_matrix

array([[ 0.        , 11.21965745, 11.12189113, 14.06334694, 19.57949189],
       [11.21965745,  0.        ,  0.73489825,  5.08672372,  9.59918209],
       [11.12189113,  0.73489825,  0.        ,  4.49055299, 10.16821689],
       [14.06334694,  5.08672372,  4.49055299,  0.        , 12.04485454],
       [19.57949189,  9.59918209, 10.16821689, 12.04485454,  0.        ]])

In [6]:
def totalCost(mask, pos, n, cost, info_lists):
    distance_list = []
    i_list = []
    # Base case: if all cities are visited, return the
    # cost to return to the starting city (0)
    if mask == (1 << n) - 1:
        return cost[pos][0]

    ans = sys.maxsize   

    # Try visiting every city that has not been visited yet
    for i in range(n):
        if (mask & (1 << i)) == 0: 
            i_list.append(i)
            # If city i is not visited, visit it and 
             #  update the mask
            distance_list.append(cost[pos][i] +
                      totalCost(mask | (1 << i), i, n, cost, info_lists))
        

    info_list = [i_list, distance_list]
    info_lists.append(info_list)
    
    ans = min(distance_list)
    return ans

In [7]:
n = len(distance_matrix)
info_list = []
optimized_distance = totalCost(1, 0, n, distance_matrix,info_list)
print("the short distance is: ", optimized_distance)

the short distance is:  47.564172952389654


In [8]:
#get a list of 1 to n
candidates = list(range(n-1))
#add 1 to the values
candidates = [x+1 for x in candidates]

moves = []

while len(candidates) > 0:
    #find the last one in the lnfo_list
    for i in range(len(info_list)):
        if info_list[i][0] == candidates:
            next_move = info_list[i][0][np.argmin(info_list[i][1])]
            moves.append(next_move)
            #take next move out of candidates
            candidates.remove(next_move)

moves_reversed = moves[::-1]
optimized_route = [[0] + moves, [0] + moves_reversed]
print(" the shortest path is: ", optimized_route)
            

 the shortest path is:  [[0, 2, 1, 3, 4], [0, 4, 3, 1, 2]]


Validation!

In [9]:
distance_matrix = [
        [0, 10, 15, 50],
        [10, 0, 50, 25],
        [15, 50, 0, 30],
        [50, 25, 30, 0]
    ]

In [10]:
n = len(distance_matrix)
info_list = []
optimized_distance = totalCost(1, 0, n, distance_matrix,info_list)
print(optimized_distance)

80


In [11]:
info_list

[[[3], [80]],
 [[2], [45]],
 [[2, 3], [130, 70]],
 [[3], [75]],
 [[1], [35]],
 [[1, 3], [125, 65]],
 [[2], [65]],
 [[1], [60]],
 [[1, 2], [90, 90]],
 [[1, 2, 3], [80, 80, 140]]]

In [12]:
#get a list of 1 to n
candidates = list(range(n-1))
#add 1 to the values
candidates = [x+1 for x in candidates]

moves = []

while len(candidates) > 0:
    #find the last one in the lnfo_list
    for i in range(len(info_list)):
        if info_list[i][0] == candidates:
            next_move = info_list[i][0][np.argmin(info_list[i][1])]
            moves.append(next_move)
            #take next move out of candidates
            candidates.remove(next_move)

moves_reversed = moves[::-1]
optimized_route = [[0] + moves + [0], [0] + moves_reversed + [0]]
print(optimized_route)

[[0, 1, 3, 2, 0], [0, 2, 3, 1, 0]]


Multi day

In [13]:
def totalCost_multiday(mask, pos, day, n, cost, info_lists):
    current_day_visits = len(cordinates[day])
    distance_list = []
    i_list = []
    # Base case: if all cities are visited, return the
    # cost to return to the starting city (0)
    if mask == (1 << n) - 1:
        return cost[pos][0]

    ans = sys.maxsize   

    # Try visiting every city that has not been visited yet
    for i in range(n):
        if (mask & (1 << i)) == 0: 
            i_list.append(i)
            # If city i is not visited, visit it and 
             #  update the mask
            distance_list.append(cost[pos][i] +
                      totalCost(mask | (1 << i), i, day, n, cost, info_lists))
        

    info_list = [i_list, distance_list]
    info_lists.append(info_list)
    
    ans = min(distance_list)
    return ans

In [14]:
cordinates

[[(39.946396818, -75.1609669488),
  (39.9488980012, -75.1500296367),
  (39.9495774962, -75.1503095688),
  (39.953893, -75.149068),
  (39.9445424967, -75.1414754636)],
 [(39.946396818, -75.1609669488),
  (39.965573, -75.180969),
  (39.942712, -75.159313),
  (39.9533414645, -75.1588545174),
  (39.9582109, -75.1731373822)]]

In [15]:
cordinates[0][0]

(39.946396818, -75.1609669488)

In [16]:
cordinates_all_days = []
for i in range(len(cordinates)):
    for business in cordinates[i]:
        cordinates_all_days.append(business)

cordinates_all_days

[(39.946396818, -75.1609669488),
 (39.9488980012, -75.1500296367),
 (39.9495774962, -75.1503095688),
 (39.953893, -75.149068),
 (39.9445424967, -75.1414754636),
 (39.946396818, -75.1609669488),
 (39.965573, -75.180969),
 (39.942712, -75.159313),
 (39.9533414645, -75.1588545174),
 (39.9582109, -75.1731373822)]

In [17]:
distance_matrix = getDistanceMatrix(cordinates_all_days)
distance_matrix

array([[ 0.        , 11.21965745, 11.12189113, 14.06334694, 19.57949189,
         0.        , 27.70934875,  4.03898878,  7.25882094, 16.96148527],
       [11.21965745,  0.        ,  0.73489825,  5.08672372,  9.59918209,
        11.21965745, 35.14683181, 11.15560151,  9.88042942, 24.91381115],
       [11.12189113,  0.73489825,  0.        ,  4.49055299, 10.16821689,
        11.12189113, 34.58116342, 11.32240308,  9.33721607, 24.40583385],
       [14.06334694,  5.08672372,  4.49055299,  0.        , 12.04485454,
        14.06334694, 33.97199142, 15.16491958,  9.80204643, 24.45361773],
       [19.57949189,  9.59918209, 10.16821689, 12.04485454,  0.        ,
        19.57949189, 44.74395475, 17.93121365, 19.47956225, 34.48626304],
       [ 0.        , 11.21965745, 11.12189113, 14.06334694, 19.57949189,
         0.        , 27.70934875,  4.03898878,  7.25882094, 16.96148527],
       [27.70934875, 35.14683181, 34.58116342, 33.97199142, 44.74395475,
        27.70934875,  0.        , 31.48980243

In [18]:
import pandas as pd

# Convert the array into a DataFrame (table)
df = pd.DataFrame(distance_matrix)

# Save the DataFrame to an Excel file
df.to_excel('output.xlsx', index=False)

In [19]:
def getHotelIndex(day,cordinates):
    hotel_index = 0
    if day > 0:
        for j in range(day):
            hotel_index += len(cordinates[j])
    return hotel_index

def totalCost_multiday(mask, pos, day, cordinates, n, visited, cost, info_lists):
    visit_requirement = len(cordinates[day])
    distance_list = []
    i_list = []

    hotel_index = getHotelIndex(day,cordinates)
    # Base case: if all cities are visited, return the
    # cost to return to the starting city (0)

    if mask == (1 << n) - 1:
        return cost[pos][hotel_index]

    if visit_requirement == visited:
        for i in range(n):
            if (mask & (1 << i)) == 0: 
                i_list.append(i)
                distance_list.append(cost[pos][i] + totalCost_multiday(mask | (1 << i), hotel_index, day + 1, cordinates, n, 1, cost, info_lists))
        
        info_list = [pos,i_list, distance_list]
        info_lists.append(info_list)
        
        return min(distance_list) + cost[pos][hotel_index] # change this to the old hotel position
    
    # Try visiting every city that has not been visited yet
    for i in range(n):
        if (mask & (1 << i)) == 0: 

            i_list.append(i)
            # If city i is not visited, visit it and 
             #  update the mask
            distance_list.append(cost[pos][i] +
                      totalCost_multiday(mask | (1 << i), i, day, cordinates, n, visited + 1, cost, info_lists))
        

    info_list = [pos,i_list, distance_list]
    info_lists.append(info_list)
  
    return min(distance_list)

In [20]:
n = len(distance_matrix)
info_list = []
#newMask will have all the hotels as 1 before the function.
newMask = 1
index_list = list(range(len(cordinates) - 1))
index_list = index_list[::-1]
newMask = 1
for i in index_list:
    newMask = (newMask << (len(cordinates[i]))) + 1

optimized_distance = totalCost_multiday(newMask,0,0,cordinates,n,1,distance_matrix, info_list)

In [21]:
optimized_distance

np.float64(105.48895393859405)

This might be correct! The seperate ones are 47 + 67, and pretty optimized, but if the new number 105 seems pretty correct.

In [22]:
#test
cordinates = [[(0,0),(0,0),(0,0)], [(0,0),(0,0),(0,0),(0,0)]]
distance_matrix = [
    [0,10,12, 1000, 13, 1000,1000],
    [10,0,11,1000,1000,1000,1000],
    [12,11,0,1000,1000,1000,1000],
    [1000,1000,1000,0,14,1000,17],
    [1000,1000,1000,14,0,15,1000,],
    [1000,1000,1000,1000,15,0,16],
    [1000,1000,1000,17,1000,16,0]
]

In [23]:
n = len(distance_matrix)
info_list = []
#newMask will have all the hotels as 1 before the function.
newMask = 1
index_list = list(range(len(cordinates) - 1))
index_list = index_list[::-1]
newMask = 1
for i in index_list:
    newMask = (newMask << (len(cordinates[i]))) + 1

optimized_distance = totalCost_multiday(newMask,0,0,cordinates,n,1,distance_matrix, info_list)
optimized_distance

2061

In [24]:
n

7

In [25]:
newMask

9

In [26]:
len(cordinates[0])

3

In [27]:
getHotelIndex(1,cordinates)

3

In [28]:
info_list

[[5, [6], [33]],
 [6, [5], [1016]],
 [0, [5, 6], [1033, 2016]],
 [4, [6], [1017]],
 [6, [4], [1014]],
 [0, [4, 6], [1030, 2014]],
 [4, [5], [1015]],
 [5, [4], [29]],
 [0, [4, 5], [1028, 1029]],
 [2, [4, 5, 6], [2033, 2030, 2028]],
 [5, [6], [33]],
 [6, [5], [1016]],
 [0, [5, 6], [1033, 2016]],
 [2, [6], [1017]],
 [6, [2], [2000]],
 [0, [2, 6], [1029, 3000]],
 [2, [5], [2000]],
 [5, [2], [2000]],
 [0, [2, 5], [2012, 3000]],
 [4, [2, 5, 6], [2033, 1044, 3012]],
 [4, [6], [1017]],
 [6, [4], [1014]],
 [0, [4, 6], [1030, 2014]],
 [2, [6], [1017]],
 [6, [2], [2000]],
 [0, [2, 6], [1029, 3000]],
 [2, [4], [1014]],
 [4, [2], [2000]],
 [0, [2, 4], [1026, 2013]],
 [5, [2, 4, 6], [2030, 1044, 1042]],
 [4, [5], [1015]],
 [5, [4], [29]],
 [0, [4, 5], [1028, 1029]],
 [2, [5], [2000]],
 [5, [2], [2000]],
 [0, [2, 5], [2012, 3000]],
 [2, [4], [1014]],
 [4, [2], [2000]],
 [0, [2, 4], [1026, 2013]],
 [6, [2, 4, 5], [2028, 3012, 1042]],
 [1, [2, 4, 5, 6], [2051, 3044, 3042, 3042]],
 [5, [6], [33]],
 [6, 

In [29]:
def getHotelIndex(day,cordinates):
    hotel_index = 0
    if day > 0:
        for j in range(day):
            hotel_index += len(cordinates[j])
    return hotel_index

def totalCost_multiday(mask, pos, day, cordinates, n, visited, cost, info_lists):
    visit_requirement = len(cordinates[day])
    distance_list = []
    i_list = []

    hotel_index = getHotelIndex(day,cordinates)
    # Base case: if all cities are visited, return the
    # cost to return to the starting city (0)

    if mask == (1 << n) - 1:
        return cost[pos][hotel_index]

    if visit_requirement == visited:
        for i in range(n):
            if (mask & (1 << i)) == 0: 
                i_list.append(i)
                distance_list.append(cost[hotel_index][i] + totalCost_multiday(mask | (1 << i), i, day + 1, cordinates, n, 2, cost, info_lists))
        
        info_list = [mask,pos,i_list, distance_list,visit_requirement,visited]
        info_lists.append(info_list)
        
        return min(distance_list) + cost[pos][hotel_index] # change this to the old hotel position
    
    # Try visiting every city that has not been visited yet
    for i in range(n):
        if (mask & (1 << i)) == 0: 

            i_list.append(i)
            # If city i is not visited, visit it and 
             #  update the mask
            distance_list.append(cost[pos][i] +
                      totalCost_multiday(mask | (1 << i), i, day, cordinates, n, visited + 1, cost, info_lists))
        

    info_list = [mask,pos,i_list, distance_list,visit_requirement,visited]
    info_lists.append(info_list)
  
    return min(distance_list)

#test
cordinates = [[(0,0),(0,0),(0,0)], [(0,0),(0,0),(0,0),(0,0)]]
distance_matrix = [
    [0,10,12, 1000, 13, 1000,1000],
    [10,0,11,1000,1000,1000,1000],
    [12,11,0,1000,1000,1000,1000],
    [1000,1000,1000,0,14,1000,17],
    [13,1000,1000,14,0,15,1000,],
    [1000,1000,1000,1000,15,0,16],
    [1000,1000,1000,17,1000,16,0]
]

n = len(distance_matrix)
info_list = []
#newMask will have all the hotels as 1 before the function.
newMask = 1
index_list = list(range(len(cordinates) - 1))
index_list = index_list[::-1]
newMask = 1
for i in index_list:
    newMask = (newMask << (len(cordinates[i]))) + 1

optimized_distance = totalCost_multiday(newMask,0,0,cordinates,n,1,distance_matrix, info_list)
info_list

[[63, 5, [6], [33], 4, 3],
 [95, 6, [5], [1016], 4, 3],
 [31, 4, [5, 6], [48, 2016], 4, 2],
 [63, 4, [6], [1017], 4, 3],
 [111, 6, [4], [1014], 4, 3],
 [47, 5, [4, 6], [1032, 1030], 4, 2],
 [95, 4, [5], [1015], 4, 3],
 [111, 5, [4], [29], 4, 3],
 [79, 6, [4, 5], [2015, 45], 4, 2],
 [15, 2, [4, 5, 6], [61, 2030, 1045], 3, 3],
 [63, 5, [6], [33], 4, 3],
 [95, 6, [5], [1016], 4, 3],
 [31, 2, [5, 6], [1033, 2016], 4, 2],
 [63, 2, [6], [1017], 4, 3],
 [123, 6, [2], [2000], 4, 3],
 [59, 5, [2, 6], [2017, 2016], 4, 2],
 [95, 2, [5], [2000], 4, 3],
 [123, 5, [2], [2000], 4, 3],
 [91, 6, [2, 5], [3000, 2016], 4, 2],
 [27, 4, [2, 5, 6], [1045, 3016, 3016], 3, 3],
 [63, 4, [6], [1017], 4, 3],
 [111, 6, [4], [1014], 4, 3],
 [47, 2, [4, 6], [2017, 2014], 4, 2],
 [63, 2, [6], [1017], 4, 3],
 [123, 6, [2], [2000], 4, 3],
 [59, 4, [2, 6], [2017, 3000], 4, 2],
 [111, 2, [4], [1014], 4, 3],
 [123, 4, [2], [2000], 4, 3],
 [107, 6, [2, 4], [2014, 3000], 4, 2],
 [43, 5, [2, 4, 6], [2026, 2030, 3014], 3, 3]

In [30]:
# Python program to find the shortest possible route
# that visits every city exactly once and returns to
# the starting point using memoization and bitmasking

def totalCost(mask, curr, n, cost, memo):
  
    # Base case: if all cities are visited, return the
    # cost to return to the starting city (0)
    if mask == (1 << n) - 1:
        return cost[curr][0]

    # If the value has already been computed, return it
    # from the memo table
    if memo[curr][mask] != -1:
        return memo[curr][mask]

    ans = float('inf')

    # Try visiting every city that has not been visited yet
    for i in range(n):
        if (mask & (1 << i)) == 0: 
          
          # If city i is not visited
          # Visit city i and update the mask
            ans = min(ans, cost[curr][i] +
                      totalCost(mask | (1 << i), i, n, cost, memo))

    # Memoize the result
    memo[curr][mask] = ans
    return ans


def tsp(cost):
    n = len(cost)
    
    # Initialize memoization table with -1
    # (indicating uncomputed states)
    memo = [[-1] * (1 << n) for _ in range(n)]

    # Start from city 0, with only city 0 visited initially (mask = 1)
    return totalCost(1, 0, n, cost, memo)

 
cost = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]
res = tsp(cost)
print(res)

80
